# report

In [187]:
import pandas as pd
import os

In [83]:
file_list=['spy/'+i for i in os.listdir('spy')]

In [84]:
sample = pd.read_csv(file_list[0],skipinitialspace=True)

In [285]:
import pandas as pd
import os

def contract_selct(
    selct_type: str = 'main',
    contract_type: str = 'C',
    data : str = 'spy',
    date: str = None,
    main: pd.DataFrame = None,
    maxDTE: int = 20,
    minDTE: int = 10,
    maxDistance: float = None,
    minDistance: float = None,
    maxTheta: float = None,
    minTheta: float = None,
    moneyness : str = 'out') ->pd.DataFrame:
    
    """
    return a dataframe containing the information of selected contracts
    
    input:
    selct_type: "main" or "hedge", declare what type of contracts we are searching, whether the main contracts we are shorting, or the contracts that we used to hedge
    contract_type: 'C' or 'P'
    date: str in form of "yyyymmdd", declare the date
    data : 'spy' or 'spx'
    main: None when selct_type = 'main', a dataframe containing the main contract when selct_type = 'hedge'
    maxDTE: max DTE
    minDTE: min DTE
    maxDistance: max distance of strike from underlying in percentage
    minDistance: min distance of strike from underlying in percentage
    maxTheta: max Theta
    minTheta: min Theta
    moneyness: 'out' or 'in'
    """
    
    if maxDistance == None: maxDistance = 1000
    if minDistance == None: minDistance = 0
    if maxTheta == None: maxTheta = 0
    if minTheta == None: minTheta = -100
    
    contract_pd = pd.DataFrame()
    
    if selct_type == 'main':
        contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                           index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
        contract_pd = contract_pd.drop(columns='Unnamed: 0')
        contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
        contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]
        if contract_type == 'C':
            contract_pd = contract_pd.drop(columns=['[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]'])
            if moneyness == 'out':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
            if moneyness == 'in':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
        if contract_type == 'P':
            contract_pd = contract_pd.drop(columns=['[C_BID]', '[C_ASK]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]'])
            if moneyness == 'out':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
            if moneyness == 'in':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
    
#         contract_pd['spread_pct'] = (contract_pd['['+contract_type+'_ASK]']-contract_pd['['+contract_type+'_BID]'])*2/(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])

#         for i in ['['+contract_type+'_DELTA]', '['+contract_type+'_GAMMA]', '['+contract_type+'_VEGA]', '['+contract_type+'_THETA]']:
#             contract_pd['price_to_'+i]=(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])/(2*contract_pd[i])

#         # utility function to select main contract    
#         contract_pd['utility'] = 5000*contract_pd['spread_pct'] + 0.5*contract_pd['price_to_['+contract_type+'_DELTA]'] -0.005*contract_pd['price_to_['+contract_type+'_GAMMA]'] - contract_pd['price_to_['+contract_type+'_VEGA]'] -50* contract_pd['price_to_['+contract_type+'_THETA]']

#         contract_pd = contract_pd.drop(columns=['spread_pct', 'price_to_['+contract_type+'_DELTA]', 'price_to_['+contract_type+'_GAMMA]', 'price_to_['+contract_type+'_VEGA]', 'price_to_['+contract_type+'_THETA]'])

#     return contract_pd.sort_values(by=['utility']).head(3)
    return contract_pd

In [286]:
contract_selct(date= '20200910')

[UNDERLYING_LAST]  [DTE]  [C_DELTA]  [C_GAMMA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                   
2020-09-10   2020-09-21                333.92   11.0    0.48860    0.02779   
             2020-09-21                333.92   11.0    0.46244    0.02709   
             2020-09-21                333.92   11.0    0.43468    0.02697   
             2020-09-21                333.92   11.0    0.40580    0.02724   
             2020-09-21                333.92   11.0    0.37705    0.02682   
...                                       ...    ...        ...        ...   
             2020-09-30                333.92   20.0    0.00258    0.00036   
             2020-09-30                333.92   20.0    0.00042    0.00009   
             2020-09-30                333.92   20.0    0.00067    0.00014   
             2020-09-30                333.92   20.0    0.00235    0.00028   
             2020-09-30                333.92   20.0    0.00094    0.00013   

                            [C_VEGA]  [C_THETA]  [C_BID]  [C_ASK]  [STRIKE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                    
2020-09-10   2020-09-21      0.24014   -0.21920     5.39     5.47     334.0   
             2020-09-21      0.24013   -0.22436     5.04     5.13     335.0   
             2020-09-21      0.23798   -0.22146     4.59     4.64     336.0   
             2020-09-21      0.23409   -0.21359     4.03     4.09     337.0   
             2020-09-21      0.22994   -0.20854     3.60     3.69     338.0   
...                              ...        ...      ...      ...       ...   
             2020-09-30      0.00633   -0.00305     0.01     0.02     405.0   
             2020-09-30      0.00275   -0.00161     0.00     0.01     410.0   
             2020-09-30      0.00207   -0.00109     0.00     0.01     415.0   
             2020-09-30      0.00553   -0.00337     0.00     0.02     420.0   
             2020-09-30      0.00165   -0.00100     0.00     0.02     425.0   

                            [STRIKE_DISTANCE]  [STRIKE_DISTANCE_PCT]  
[QUOTE_DATE] [EXPIRE_DATE]                                            
2020-09-10   2020-09-21                   0.1                  0.000  
             2020-09-21                   1.1                  0.003  
             2020-09-21                   2.1                  0.006  
             2020-09-21                   3.1                  0.009  
             2020-09-21                   4.1                  0.012  
...                                       ...                    ...  
             2020-09-30                  71.1                  0.213  
             2020-09-30                  76.1                  0.228  
             2020-09-30                  81.1                  0.243  
             2020-09-30                  86.1                  0.258  
             2020-09-30                  91.1                  0.273  

[295 rows x 11 columns]

In [275]:
data= 'spx'
date= '20200910'
maxDTE: int = 30,
minDTE: int = 10,
maxDistance: float = 0.2,
minDistance: float = 0.01,
maxTheta: float = -0.0,
minTheta: float = -1.2,
contract_type: str = 'P'
moneyness = 'out'

contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                   index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
contract_pd = contract_pd.drop(columns='Unnamed: 0')
contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]

if contract_type == 'C':
    contract_pd = contract_pd.drop(columns=['[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]'])
    if moneyness == 'out':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
    if moneyness == 'in':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
if contract_type == 'P':
    contract_pd = contract_pd.drop(columns=['[C_BID]', '[C_ASK]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]'])
    if moneyness == 'out':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
    if moneyness == 'in':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]



In [280]:
contract_pd['spread_pct'] = (contract_pd['['+contract_type+'_ASK]']-contract_pd['['+contract_type+'_BID]'])*2/(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])

for i in ['['+contract_type+'_DELTA]', '['+contract_type+'_GAMMA]', '['+contract_type+'_VEGA]', '['+contract_type+'_THETA]']:
    contract_pd['price_to_'+i]=(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])/(2*contract_pd[i])

# utility function to select main contract    
contract_pd['utility'] = 5000*contract_pd['spread_pct'] + 0.5*contract_pd['price_to_['+contract_type+'_DELTA]'] -0.005*contract_pd['price_to_['+contract_type+'_GAMMA]'] - contract_pd['price_to_['+contract_type+'_VEGA]'] -50* contract_pd['price_to_['+contract_type+'_THETA]']

contract_pd = contract_pd.drop(columns=['spread_pct', 'price_to_['+contract_type+'_DELTA]', 'price_to_['+contract_type+'_GAMMA]', 'price_to_['+contract_type+'_VEGA]', 'price_to_['+contract_type+'_THETA]'])

return contract_pd.sort_values(by=['utility']).head(3)